In [ ]:

############################################################## HCA 진행 ##########################################################################

!pip install rdkit-pypi

import pandas as pd
import matplotlib.pyplot as plt
from rdkit import Chem
from rdkit.Chem import rdFingerprintGenerator
from scipy.cluster.hierarchy import dendrogram, linkage
from google.colab import files


# smiles 정보 담긴 파일 업로드
uploaded = files.upload()
filename = next(iter(uploaded))  # 업로드된 파일 이름 가져오기

# Excel 파일로부터 필요한 정보 빼내기
df = pd.read_excel(filename)
labels= df[df.columns[0]].tolist()
smiles_list= df[df.columns[2]].tolist()
groups= df[df.columns[3]].tolist()
groups = [group.strip() for group in groups]  # 공백 제거


#  RDKit을 사용하기 위한 smils ==> Mol 변환 및 Mol객체를 사용한 topological fingerprint 추출
molecule_list = [Chem.MolFromSmiles(x) for x in smiles_list]
gen = rdFingerprintGenerator.GetRDKitFPGenerator()
fingerprints = [gen.GetFingerprint(x) for x in molecule_list]

# Fingerprint 배열 생성
fp_array = [list(fp) for fp in fingerprints]

# HCA 수행
Z = linkage(fp_array, method='ward', metric = 'euclidean')

# dendrogram 그리기
plt.figure(figsize=(10, 8))
dendrogram(Z, labels=labels, orientation='top', leaf_rotation=90, leaf_font_size=10,
           distance_sort='descending', show_leaf_counts=True, color_threshold=40,
           above_threshold_color="grey")
plt.title("Hierarchical Clustering Dendrogram", fontsize=20, weight='bold')
plt.gca().spines['left'].set_visible(False)
plt.gca().spines['top'].set_visible(False)
plt.gca().spines['right'].set_visible(False)
plt.show()



In [ ]:
############################################################## PCA 진행 ##########################################################################

!pip install scikit-learn

import pandas as pd
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA

# PCA 모델 생성 및 학습 (n_components= 추출할 주성분의 수, 이전에 불러온 파일의 fp_array 사용하여 PCA 진행)
pca = PCA(n_components=2)
principal_components = pca.fit_transform(fp_array)

# 주성분으로 이루어진 데이터 프레임 생성
pca_df = pd.DataFrame(data=principal_components, columns=['PC1', 'PC2'])

# 그룹별 색상 매핑
group_colors = {
    'Phenidate': 'purple',
    '2C Series': 'red',
    'PCP': 'yellow',
    'Methcathinone': 'green',
    'Benzodiazepine': 'brown'
}
colors = [group_colors.get(group, 'gray') for group in groups]  # 없는 그룹일 경우 'gray' 사용

# 결과 시각화
plt.figure(figsize=(10, 8))
plt.scatter(pca_df['PC1'], pca_df['PC2'], c=colors, alpha=0.7)
for i, txt in enumerate(labels):
    plt.annotate(txt, (pca_df['PC1'][i], pca_df['PC2'][i]), fontsize=5)

plt.title('PCA Result')
plt.xlabel('Principal Component 1')
plt.ylabel('Principal Component 2')
plt.grid(False)
plt.show()

# 각 주성분의 설명된 분산 비율 출력
print("Explained variance ratio:", pca.explained_variance_ratio_)
